In [6]:
import sys
sys.path.append('../..')

import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import torch.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.metrics.functional import accuracy
import torchdiffeq

from torchdyn.models import *; from torchdyn import *

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [22]:
batch_size=64
size=28
path_to_data='../../data/mnist_data'

all_transforms = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

train_data = datasets.MNIST(path_to_data, train=True, download=True,
                            transform=all_transforms)
test_data = datasets.MNIST(path_to_data, train=False,
                           transform=all_transforms)

trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

## Train loop

In [71]:
class Learner(pl.LightningModule):
    def __init__(self, model:nn.Module):
        super().__init__()
        self.lr = 1e-3
        self.model = model
        self.iters = 0
        
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        self.iters += 1
        x, y = batch   
        x, y = x.to(device), y.to(device)
        y_hat = self.model(x)   
        loss = nn.CrossEntropyLoss()(y_hat, y)
        
        acc = accuracy(y_hat, y)
        nfe = model[2].nfe ; model[2].nfe = 0
        tqdm_dict = {'train_loss': loss, 'accuracy': acc, 'NFE': nfe}
        
        self.logger.experiment.log(step=self.iters, row=tqdm_dict)
        return {'loss': loss}   

    def test_step(self, batch, batch_nb):
        x, y = batch
        x, y = x.to(device), y.to(device)
        y_hat = self(x)
        test_acc = accuracy(y_hat, y)
        test_loss = nn.CrossEntropyLoss()(y_hat, y)
        metrics = {'test_loss': test_loss, 'test_accuracy': test_acc}
        self.logger.experiment.log(step=self.iters, row=metrics)
        return {'test_loss': test_loss}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['test_accuracy'] for x in outputs]).mean()
        metrics = {'avg_test_loss': avg_loss, 'avg_test_accuracy': avg_acc}
        self.logger.experiment.log(step=self.iters, row=metrics)
        return {'avg_test_loss': avg_loss}
    
    def validation_step(self, batch, batch_nb):
        x, y = batch
        x, y = x.to(device), y.to(device)
        y_hat = self(x)
        val_acc = accuracy(y_hat, y)
        val_loss = nn.CrossEntropyLoss()(y_hat, y)
        metrics = {'val_loss': val_loss, 'val_accuracy': val_acc}
        self.logger.experiment.log(step=self.iters, row=metrics)
        return {'val_loss': val_loss, 'val_accuracy': val_acc}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_accuracy'] for x in outputs]).mean()
        metrics = {'avg_val_loss': avg_loss, 'avg_val_accuracy': avg_acc}
        self.logger.experiment.log(step=self.iters, row=metrics)
        return {'avg_val_loss': avg_loss}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=1e-4)

    def train_dataloader(self):
        self.trloader_len = len(trainloader)
        return trainloader
    
    def val_dataloader(self):
        self.vloader_len = len(testloader) # using test as val here
        return testloader
    
    def test_dataloader(self):
        self.test_tloader_len = len(testloader)
        return testloader

## Model

In [77]:
func = nn.Sequential(DataControl(),
                     nn.Conv2d(12, 6, 3, padding=1),
                     nn.Tanh(),
                     ).to(device)

neuralDE = NeuralDE(func, 
                   solver='rk4',
                   sensitivity='adjoint',
                   s_span=torch.linspace(0, 1, 30)).to(device)

model = nn.Sequential(Augmenter(augment_dims=5),
                      nn.BatchNorm2d(6),
                      neuralDE,
                      nn.Conv2d(6, 1, 3, padding=1),
                      nn.Flatten(),
                      nn.Dropout(p=0.5),
                      nn.Linear(28*28, 10)).to(device)




In [80]:
logger = WandbLogger(project='torchdyn-mnist-bench', log_model=True) # feel free to comment out or use a different logging scheme :)

In [81]:
learn = Learner(model)
trainer = pl.Trainer(max_epochs=20,
                     logger=logger,
                     benchmark=True,
                     #limit_test_batches=0.25,
                     val_check_interval=0.05,
                     gpus=1,  
                     progress_bar_refresh_rate=1
                     )

trainer.fit(learn)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]



  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 8 K   
/home/jyp/michael_dev/testenv/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Your val_dataloader has shuffle=True, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)
/home/jyp/michael_dev/testenv/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/jyp/michael_dev/testenv/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/jyp/michael_dev/testenv/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: The metric you returned None must be a `torch.Tensor` instance, checkpoint not saved HINT: what is the value of val_loss in validation_epoch_end()?
  warnings.warn(*args, **kwargs)
/home/jyp/michael_dev/testenv/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: Can save best model only with val_loss available, skipping.
  warnings.warn(*args, **kwargs)


/home/jyp/michael_dev/testenv/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [66]:
trainer.test(learn)

/home/jyp/michael_dev/testenv/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Your test_dataloader has shuffle=True, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)
/home/jyp/michael_dev/testenv/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


wandb: WARNING Adding to old History rows isn't currently supported.  Step 18760 < 18761; dropping {'test_loss': tensor(0.0424, device='cuda:0'), 'test_accuracy': tensor(1., device='cuda:0')}.
wandb: WARNING Adding to old History rows isn't currently supported.  Step 18760 < 18761; dropping {'test_loss': tensor(0.0958, device='cuda:0'), 'test_accuracy': tensor(0.9219, device='cuda:0')}.
wandb: WARNING Adding to old History rows isn't currently supported.  Step 18760 < 18761; dropping {'test_loss': tensor(0.0706, device='cuda:0'), 'test_accuracy': tensor(0.9844, device='cuda:0')}.
wandb: WARNING Adding to old History rows isn't currently supported.  Step 18760 < 18761; dropping {'test_loss': tensor(0.0696, device='cuda:0'), 'test_accuracy': tensor(0.9844, device='cuda:0')}.
wandb: WARNING Adding to old History rows isn't currently supported.  Step 18760 < 18761; dropping {'test_loss': tensor(0.0306, device='cuda:0'), 'test_accuracy': tensor(1., device='cuda:0')}.
wandb: WARNING Adding t

--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_accuracy': tensor(0.9802, device='cuda:0'),
 'avg_test_loss': tensor(0.0580, device='cuda:0')}
--------------------------------------------------------------------------------

